In [34]:
!pip install gradio diagnostipy


In [35]:
import gradio as gr
import math
from diagnostipy.core.models.symptom_rule import SymptomRule
from diagnostipy.core.ruleset import SymptomRuleset
from diagnostipy.core.evaluator import Evaluator
from diagnostipy.utils.enums import EvaluationFunctionEnum, ConfidenceFunctionEnum

In [36]:
rules = [
    SymptomRule(
        name="High Fever",
        weight=7,
        critical=False,
        conditions={"fever"},
        apply_condition=lambda data: data.get("fever", 0) >= 38.0,
    ),
    SymptomRule(
        name="Moderate Fever",
        weight=3,
        critical=False,
        conditions={"fever"},
        apply_condition=lambda data: 37.5 <= data.get("fever", 0) < 38.0,
    ),
    SymptomRule(
        name="Persistent Cough",
        weight=8,
        critical=False,
        conditions={"cough"},
    ),
    SymptomRule(
        name="Sore Throat",
        weight=4,
        critical=False,
        conditions={"sore_throat"},
    ),
    SymptomRule(
        name="Fatigue",
        weight=3,
        critical=False,
        conditions={"fatigue"},
    ),
    SymptomRule(
        name="High Fever with Fatigue",
        weight=10,
        critical=True,
        apply_condition=lambda data: data.get("fever", 0) >= 38.0
        and data.get("fatigue", False),
        conditions={"fever", "fatigue"},
    ),
]

evaluator_settings = {
    "exclude_overlaps": True,
    "evaluation_function": EvaluationFunctionEnum.BINARY_SIMPLE,
    "confidence_function": ConfidenceFunctionEnum.WEIGHTED,
    "score_function": None,
    "score_threshold": None,
    "labels": None,
    "threshold_label_map": None,
}
ruleset = SymptomRuleset(rules=rules, exclude_overlaps=evaluator_settings["exclude_overlaps"])

In [42]:
def update_evaluator_settings(data: dict):
    ruleset.exclude_overlaps = data["exclude_overlaps"]
    evaluator_settings["evaluation_function"] = data["evaluation_function"]
    evaluator_settings["confidence_function"] = data["confidence_function"]

def show_options(evaluation_function_value):
    visibility = {
        "score_function": False,
        "score_threshold": False,
        "labels": False,
        "threshold_label_map": False,
    }
    if evaluation_function_value == EvaluationFunctionEnum.BINARY_SCORING_BASED.value:
        visibility["score_function"] = True
        visibility["score_threshold"] = True
    elif evaluation_function_value == EvaluationFunctionEnum.MULTICLASS_SIMPLE.value:
        visibility["labels"] = True
    elif evaluation_function_value == EvaluationFunctionEnum.MULTICLASS_SCORING_BASED.value:
        visibility["labels"] = True
        visibility["threshold_label_map"] = True

    return (
        gr.update(visible=visibility["score_function"]),
        gr.update(visible=visibility["score_threshold"]),
        gr.update(visible=visibility["labels"]),
        gr.update(visible=visibility["threshold_label_map"]),
    )

SCORE_FUNCTIONS = {
    "identity": lambda _: lambda score: score,
    "scale_by_0.1": lambda _: lambda score: score / 10,
    "tanh": lambda _: lambda score: math.tanh(score / 10),
    "sigmoid": lambda _: lambda score: 1 / (1 + math.exp(-score / 10)),
}
def get_score_function(func_name):
    score_function_factory = SCORE_FUNCTIONS.get(func_name)
    if score_function_factory is None:
        raise ValueError(f"Invalid score function: {func_name}")
    return score_function_factory(None)  


In [43]:
def flu_diagnosis_demo(data, score_function=None, score_threshold=None, labels=None, threshold_label_map=None):
    evaluator = Evaluator(
        ruleset,
        data=data,
        evaluation_function=evaluator_settings["evaluation_function"],
        confidence_function=evaluator_settings["confidence_function"],
    )
    
    score_function_callable = None
    if score_function is not None:
        score_function_callable = get_score_function(score_function)
    
    evaluator.evaluate(
        score_function=score_function_callable,
        score_threshold=score_threshold,
        labels=labels,
        threshold_label_map=threshold_label_map,
    )
    applicable_rules = ruleset.get_applicable_rules(data)
    result = evaluator.get_results()

    output = {
        "Diagnosis": result.label if hasattr(result, "label") else "N/A",
        "Total Score": result.total_score if hasattr(result, "total_score") else "N/A",
        "Confidence": result.confidence if hasattr(result, "confidence") else "N/A",
        "Applicable Rules": [rule.name for rule in applicable_rules],
    }
    return output

In [44]:
def diagnosis_tab():
    with gr.Tab("Diagnosis"):
        gr.Markdown("# Flu Diagnosis with Diagnostipy - Interactive Demo")
        fever = gr.Number(label="What is your temperature? (°C)", value=36.6, interactive=True)
        cough = gr.Radio(choices=["True", "False", "Not Given"], label="Do you have a persistent cough?", value="Not Given")
        sore_throat = gr.Radio(choices=["True", "False", "Not Given"], label="Do you have a sore throat?", value="Not Given")
        fatigue = gr.Radio(choices=["True", "False", "Not Given"], label="Do you feel fatigued?", value="Not Given")
        result_output = gr.JSON(label="Results")
        evaluate_button = gr.Button("Evaluate")

        def gather_inputs(f, c, s, fa):
            def parse_bool(value):
                if value == "True":
                    return True
                elif value == "False":
                    return False
                return None 

            return {
                "fever": f,
                "cough": parse_bool(c),
                "sore_throat": parse_bool(s),
                "fatigue": parse_bool(fa),
            }

        evaluate_button.click(
            lambda f, c, s, fa: flu_diagnosis_demo(
                gather_inputs(f, c, s, fa),
                score_function=evaluator_settings.get("score_function"),
                score_threshold=evaluator_settings.get("score_threshold"),
                labels=evaluator_settings.get("labels"),
                threshold_label_map=evaluator_settings.get("threshold_label_map"),
            ),
            inputs=[fever, cough, sore_throat, fatigue],
            outputs=result_output,
        )


In [45]:
def settings_tab():
    with gr.Tab("Settings"):
        gr.Markdown("# Configure Rules and Evaluator Settings")

        exclude_overlaps = gr.Checkbox(label="Exclude Overlapping Rules", value=True)
        evaluation_function = gr.Dropdown(
            choices=[e.value for e in EvaluationFunctionEnum],
            label="Evaluation Function",
            value=EvaluationFunctionEnum.BINARY_SIMPLE.value,
            interactive=True,
        )
        confidence_function = gr.Dropdown(
            choices=[c.value for c in ConfidenceFunctionEnum],
            label="Confidence Function",
            value=ConfidenceFunctionEnum.WEIGHTED.value,
            interactive=True,
        )

        score_function = gr.Dropdown(
            choices=["identity", "scale_by_0.1", "tanh", "sigmoid"],
            label="Score Function",
            interactive=True,
            visible=False,
        )
        score_threshold = gr.Number(
            label="Score Threshold",
            value=0.5,
            interactive=True,
            visible=False,
        )
        labels = gr.Textbox(
            label="Labels (comma-separated for multiclass)",
            value="Low,Medium,High",
            interactive=True,
            visible=False,
        )
        threshold_label_map = gr.Textbox(
            label="Threshold-Label Map (JSON format for multiclass)",
            value='{"0.5": "Low", "1.0": "Medium", "1.5": "High"}',
            interactive=True,
            visible=False,
        )

        evaluation_function.change(
            show_options,
            inputs=[evaluation_function],
            outputs=[score_function, score_threshold, labels, threshold_label_map],
        )

        update_settings_button = gr.Button("Update Settings")

        def handle_update(eo, ef, cf, sf, st, lbls, tlm):
            evaluator_settings.update({
                "exclude_overlaps": eo,
                "evaluation_function": EvaluationFunctionEnum(ef),
                "confidence_function": ConfidenceFunctionEnum(cf),
                "score_function": sf,
                "score_threshold": st,
                "labels": lbls.split(",") if lbls else None,
                "threshold_label_map": eval(tlm) if tlm else None,
            })

        update_settings_button.click(
            handle_update,
            inputs=[
                exclude_overlaps,
                evaluation_function,
                confidence_function,
                score_function,
                score_threshold,
                labels,
                threshold_label_map,
            ],
            outputs=None,
        )

In [46]:
with gr.Blocks() as demo:
    diagnosis_tab()
    settings_tab()

demo.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
